In [28]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn 


In [29]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [30]:
hm_epochs = 10
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28

rnn_size = 128

# height * width
x = tf.placeholder('float', [None, n_chunks, chunk_size])
y = tf.placeholder('float')

In [37]:
def recurrent_neural_network(x):
    # (input_data * weights) + biases
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size, n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1 , chunk_size])
    x = tf.split(x, n_chunks, 0)
    
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']
    
    return output
    

In [38]:
def train_nerual_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    
    #learning_rate = 0.001 default rate
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    #Cycles of feed forward and backprops
   
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))
                
                _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))

In [39]:
train_nerual_network(x)

Epoch 0 completed out of 10 loss 210.487624146
Epoch 1 completed out of 10 loss 56.9654835127
Epoch 2 completed out of 10 loss 38.7683143308
Epoch 3 completed out of 10 loss 30.0564171234
Epoch 4 completed out of 10 loss 25.1428762134
Epoch 5 completed out of 10 loss 21.2228403646
Epoch 6 completed out of 10 loss 18.9076181594
Epoch 7 completed out of 10 loss 16.219289185
Epoch 8 completed out of 10 loss 14.1028318072
Epoch 9 completed out of 10 loss 13.0735249389
Accuracy: 0.9851
